In [1]:
from qbraid import device_wrapper, get_devices
from qbraid.devices._utils import SUPPORTED_VENDORS

In [3]:
get_devices()

Provider      Device Name                     Num Qubits    qBraid ID
------------  ------------------------------  ------------  ---------------------------
AWS           State Vector Simulator          --            aws_native_sv_sim
AWS           Density Matrix simulator        --            aws_native_dm_sim
AWS           Tensor Network simulator        --            aws_native_tn_sim
AWS           Braket Default Simulator        --            aws_braket_default_sim
D-Wave        2000Q-6 QPU                     2041          aws_dwave_2000Q_6
D-Wave        Advantage1.1 QPU                5436          aws_dwave_advantage_system1
IonQ          IonQ QPU                        11            aws_ionQ
Rigetti       Aspen-9 QPU                     32            aws_rigetti_aspen9
Google        Cirq Sparse Simulator           --            google_cirq_sparse_sim
Google        Cirq Density Matrix Simulator   --            google_cirq_dm_sim
IBM Quantum   Armonk QPU                      1  

In [4]:
SUPPORTED_VENDORS

{'aws': {'aws': {'aws_native_sv_sim': 'arn:aws:braket:::device/quantum-simulator/amazon/sv1',
   'aws_native_dm_sim': 'arn:aws:braket:::device/quantum-simulator/amazon/dm1',
   'aws_native_tn_sim': 'arn:aws:braket:::device/quantum-simulator/amazon/tn1'},
  'braket': {'aws_braket_default_sim': <braket.devices.local_simulator.LocalSimulator at 0x1080595e0>},
  'dwave': {'aws_dwave_2000Q_6': 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6',
   'aws_dwave_advantage_system1': 'arn:aws:braket:::device/qpu/d-wave/Advantage_system1'},
  'ionQ': {'aws_ionQ': 'arn:aws:braket:::device/qpu/ionq/ionQdevice'},
  'rigetti': {'aws_rigetti_aspen9': 'arn:aws:braket:::device/qpu/rigetti/Aspen-9'}},
 'google': {'aqt': {},
  'google': {},
  'cirq': {'google_cirq_sparse_sim': <cirq.sim.sparse_simulator.Simulator at 0x154d072b0>,
   'google_cirq_dm_sim': <cirq.sim.density_matrix_simulator.DensityMatrixSimulator at 0x154d07c40>},
  'ionQ': {},
  'pasqal': {}},
 'ibm': {'q': {'ibm_q_armonk': 'ibmq_armonk',
   '

In [2]:
import cirq
import numpy as np
from braket.circuits import Circuit as BraketCircuit
from qiskit import QuantumCircuit as QiskitCircuit
import networkx as nx
import dwave_networkx as dnx

In [5]:
n = 5
m = 10
graph = nx.gnm_random_graph(n, m, seed=42)

In [6]:
# result = dnx.min_vertex_cover(graph, dwave_sampler, answer_mode="histogram")
# print('Result to MVC problem:', result)
# print('Size of the vertex cover:', len(result))

Result to MVC problem: [0, 1, 2, 4]
Size of the vertex cover: 4


In [34]:
def device_wrapper_inputs(vendor: str, dwave=False):
    """Returns list of tuples containing all device_wrapper inputs for given vendor."""
    input_list = []
    for provider in SUPPORTED_VENDORS[vendor]:
        if dwave == (provider == "dwave"):
            for device in SUPPORTED_VENDORS[vendor][provider]:
                input_list.append(device)
    return input_list

In [8]:
def braket_circuit():
    circuit = BraketCircuit()
    circuit.h(0)
    circuit.ry(0, np.pi / 2)
    return circuit

In [9]:
def cirq_circuit(meas=True):
    q0 = cirq.GridQubit(0, 0)
    def test_circuit():
        yield cirq.H(q0)
        yield cirq.Ry(rads=np.pi / 2)(q0)
        if meas:
            yield cirq.measure(q0, key='q0')
    circuit = cirq.Circuit()
    circuit.append(test_circuit())
    return circuit

In [10]:
def qiskit_circuit(meas=True):
    circuit = QiskitCircuit(1, 1) if meas else QiskitCircuit(1)
    circuit.h(0)
    circuit.ry(np.pi / 2, 0)
    if meas:
        circuit.measure(0, 0)
    return circuit